# Prediction of spatiotemporal field of pulsation velocity in cylindrical wake by Cascade Net

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Background

In the process of turbulent spatiotemporal evolution, the pulsating velocity field includes a series of important fluid physical processes, such as separation, transition, and energy transfer. At high Reynolds numbers, the pulsating velocity field exhibits significant nonlinear characteristics. There are vortex structures in turbulent wake that range from maximum to minimum scales, and these fluid motion patterns constitute complex flow field structural characteristics. The process of energy transfer from large-scale structures to small-scale structures in these flow field structures is called the energy cascade physics principle. Inspired by this principle, the small-scale prediction problem can be transformed into a step-by-step prediction problem from large-scale to small-scale.

## Model framework

The model framework is as shown in the following figure:

![Cascade-Net](images/Cascade-Net.png)

Where, the generator is a U-Net structure with spatial and channel attention gates, and its framework is shown in the following figure:

![The U-Net structure of the generator with spatial and channel attention gates](images/The_U-Net_structure_of_the_generator_with_spatial_and_channel_attention_gates.png)

And for the spatial attention gate *S* and channel attention gate *C*, the integration diagrams are shown as follows:

![Spatial attention gate S](images/Spatial_attention_gate_S.png)

![Channel attention gate C](images/Channel_attention_gate_C.png)

## Preparation

Before practice, ensure that MindSpore of suitable version has been correctly installed. If not, you can run the following command:

* [MindSpore installation page](https://www.mindspore.cn/install) Install MindSpore.


## Training

Import code packs.

In [10]:
import gc
import argparse
import time

import mindspore as ms
import mindspore.nn as nn
from mindspore.dataset import GeneratorDataset
from mindspore import ops, Tensor, context
from mindflow.utils import load_yaml_config
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

from src import (init_sub_model, DefineCompoundCritic, DefineCompoundGan, WassersteinLoss, GradLoss, AccesstrainDataset,
                 validation_test_dataset)

matplotlib.use('agg')  # to prevent "Fail to create pixmap with Tk_GetPixmap in TkImgPhotoInstanceSetSize"
ms.dataset.config.set_prefetch_size(1)

## Datasets Preparation

Dataset download link: [Cascade_Net/dataset](https://download-mindspore.osinfra.cn/mindscience/mindflow/dataset/applications/research/Cascade_Net/). Save the dataset under path `./dataset`.

The datasets are mat type file, each containing three dictionaries: train, validation, and test, representing training dataset, validation dataset, and test dataset. The samples number of training dataset is 10792, the samples number of validation datase is 1704, and the samples number of test dataset is 3408. The Reynolds number distribution of training set data, validation set data, and test set data is shown in the figure:

![Sample cases for training validation and testing](images/Sample_cases_for_training_validation_and_testing.png)

* **conditions_Re_pd.mat** including Reynolds number information with a dimensions(*sample*, *Re*) of (, 1), where *Re* is Reynolds number。
* **Input_flucReal_Cp_pd.mat** including spatiotemporal characteristic matrix of wall pulsating pressure with a dimensions(*sample*, *θ*, *t*, *C*) of (, 128, 128, 3), where *θ* is measurement point phase of wall pressure field, *t* is time where the time length is controlled by the characteristic period of pulsating pressure, *C* is the channel number (3 channels are 3 measuring planes of wall pulsating pressure).
* **Input_scaling_u_pd.mat** including flow field supplementary input featureswith a dimensions (*sample*, *U*) of (, 20), where *U* is supplementary input features of 10 sampling points flow field *u* and *v*.
* **ur_1.mat**，**ur_3.mat**，**ur_5.mat**，**ur_10.mat** including flow field information at four scales with a dimensions (*sample*, *H*, *W*, *C*) of (, 128, 128, 2) where *H* and *W* are  flow field resolution, *C* is the channel number(2 channels are velocity U and V).


In [11]:
data_dir = 'dataset/'
loader = AccesstrainDataset(data_dir)
train_dataset = GeneratorDataset(source=loader, column_names=[
    'u_r10_train', 'u_r5_train', 'u_r3_train', 'u_r1_train', 'cp_fluc_train', 'Re_c_train', 'scaling_input_train'])
train_dataset = train_dataset.shuffle(buffer_size=25)

(u_r10_validation, u_r5_validation, u_r3_validation, u_r1_validation, cp_fluc_validation, Re_c_validation,
 scaling_input_validation, u_r10_test, u_r5_test, u_r3_test, u_r1_test, cp_fluc_test, Re_c_test, scaling_input_test) \
    = validation_test_dataset(data_dir)

## Configure Network and Training Parameters

Load network related parameters from the configuration file.

In [12]:
def parse_args():
    """parse arguments"""
    parser = argparse.ArgumentParser(description="Cascade Net")
    parser.add_argument("--mode", type=str, default="PYNATIVE", choices=["PYNATIVE"],
                        help="This case only supports PYNATIVE_MODE")
    parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                        help="The target device to run, support 'Ascend', 'GPU'")
    parser.add_argument("--device_id", type=int, default=0,
                        help="ID of the target device")
    parser.add_argument("--config_file_path", type=str,
                        default="./config/Cascade-Net.yaml")
    result_args = parser.parse_known_args()[0]
    return result_args

input_args = parse_args()
context.set_context(mode=context.PYNATIVE_MODE,
                    device_target=input_args.device_target,
                    device_id=input_args.device_id)
config = load_yaml_config(input_args.config_file_path)

data_config = config["data"]
critic_config = config["critic"]
generator_config = config["generator"]
summary_config = config["summary"]

data_dir = data_config["root_dir"]
latent_z_n_channel = data_config["latent_z_n_channel"]
batch_size = data_config["batch_size"]
n_channel_p = data_config["n_channel_p"]
n_channel_u = data_config["n_channel_u"]
column_names = data_config["column_names"]
lambda_GP = critic_config["lambda_GP"]
critic_model_lr = critic_config["critic_model_lr"]
n_critic = critic_config["n_critic"]
lambda_L2_u = generator_config["lambda_L2_u"]
lambda_L2_gradu = generator_config["lambda_L2_gradu"]
gan_model_lr = generator_config["gan_model_lr"]
plot_n = summary_config["plot_n"]
n_imgs = summary_config["n_imgs"]
merge_n_imgs = summary_config["merge_n_imgs"]
sample_interval = summary_config["sample_interval"]
epochs = summary_config["epochs"]
dxy = summary_config["dxy"]
losslog = []

train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
sample_num_train = loader.__len__()
sample_num_val = u_r1_validation.shape[0]
sample_num_test = u_r1_test.shape[0]

## Model Building

Build sub networks at various scales using the initial_model(), including sub generators and sub discriminators at four scales, and use them in the initialization of generator and discriminator.


In [13]:
# initialization of generator and discriminator
(merge_model, g_model_I, d_model_I, g_model_II, d_model_II,
 g_model_III, d_model_III, g_model_IV, d_model_IV) = init_sub_model(n_channel_p, n_channel_u)
critic_model = DefineCompoundCritic(n_channel_p, n_channel_u, batch_size,
                                    d_model_I, d_model_II, d_model_III, d_model_IV)
critic_model.update_parameters_name('critic')
gan_model = DefineCompoundGan(n_channel_p, n_channel_u, merge_model,
                              g_model_I, g_model_II, g_model_III, g_model_IV)
gan_model.update_parameters_name('generator')

## Train Function

The main part code of model training, including the definition of the loss function, the forward calculation process of the discriminator and generator, and the training process.


In [14]:
# loss function
wasserstein_loss = WassersteinLoss()
mae_loss = nn.MAELoss(reduction='none')
grad_loss = GradLoss(dxy)

# forward calculation of discriminator
def d_forward_fn(input):
    g_input = [input[4], input[5], input[6], input[7]]
    g_pred = gan_model(g_input)
    d_input = [input[0], input[1], input[2], input[3], g_pred[4], g_pred[0], g_pred[1], g_pred[2], g_pred[3]]

    d_pred = critic_model(d_input)
    loss_wass = wasserstein_loss(ops.stack(d_pred[:8]), ops.Concat(axis=0)(
        [-ops.ones_like(ops.stack(d_pred[:4])), ops.ones_like(ops.stack(d_pred[4:8]))]))
    loss_gradp = ops.stack([d_pred[12](), d_pred[13](), d_pred[14](), d_pred[15]()])
    loss = ops.sum(1 * loss_wass) + ops.sum(lambda_GP * loss_gradp)
    loss_list = ms.numpy.concatenate((ms.numpy.expand_dims(loss, 0), loss_wass, loss_gradp), axis=0).asnumpy()
    return loss, loss_list

# forward calculation of generator
def g_forward_fn(true, input):
    g_input = [input[4], input[5], input[6], input[7]]
    g_pred = gan_model(g_input)
    d_input = [input[0], input[1], input[2], input[3], g_pred[4], g_pred[0], g_pred[1], g_pred[2], g_pred[3]]
    d_pred = critic_model(d_input)

    loss_wass = wasserstein_loss(ops.stack(d_pred[4:8]), -ops.ones_like(ops.stack(d_pred[4:8])))
    loss_mae = mae_loss(ops.stack(g_pred[:4]), ops.stack(true[:4]))
    loss_mae = ops.mean(loss_mae, axis=(1, 2, 3, 4))
    loss_grad = grad_loss(ops.stack(g_pred[5:9]), ops.stack(true[:4]))
    loss = (ops.sum(1 * loss_wass) + ops.sum(lambda_L2_u * loss_mae) + ops.sum(lambda_L2_gradu * loss_grad))
    loss_list = ms.numpy.concatenate((ms.numpy.expand_dims(loss, 0), loss_wass, loss_mae, loss_grad), axis=0).asnumpy()
    return loss, loss_list

# training process
def train_step(g_real_data, input):
    for _ in range(n_critic):
        (_, d_loss_list), d_grads = d_grad_fn(input)
        d_optimizer(d_grads)
        ave_g_loss_train_ncritic.append(d_loss_list)
    ave_d_loss_train.append(np.mean(ave_g_loss_train_ncritic, axis=0))
    (_, g_loss_list), g_grads = g_grad_fn(g_real_data, input)
    g_optimizer(g_grads)
    ave_g_loss_train.append(g_loss_list)

# evaluation process
def define_evaluation(u_r10, u_r5, u_r3, u_r1, cp, Re, scaling_input, latent_z, idx):
    _, batch_d_loss = d_forward_fn(
        [Tensor(u_r10[idx, :, :, :]), Tensor(u_r5[idx, :, :, :]), Tensor(u_r3[idx, :, :, :]), Tensor(u_r1[idx, :, :, :]),
         Tensor(cp[idx, :, :, :]), Tensor(Re[idx, :]), Tensor(scaling_input[idx, :]), latent_z])
    _, batch_g_loss = g_forward_fn(
        [Tensor(u_r10[idx, :, :, :]), Tensor(u_r5[idx, :, :, :]), Tensor(u_r3[idx, :, :, :]), Tensor(u_r1[idx, :, :, :])],
        [Tensor(u_r10[idx, :, :, :]), Tensor(u_r5[idx, :, :, :]), Tensor(u_r3[idx, :, :, :]), Tensor(u_r1[idx, :, :, :]),
         Tensor(cp[idx, :, :, :]), Tensor(Re[idx, :]), Tensor(scaling_input[idx, :]), latent_z])
    return batch_d_loss, batch_g_loss


## Optimizer

Apply the RMSProp optimizer.


In [15]:
# optimizer
d_optimizer = nn.RMSProp(critic_model.trainable_params(), learning_rate=critic_model_lr)
d_grad_fn = ms.value_and_grad(d_forward_fn, None, d_optimizer.parameters, has_aux=True)
g_optimizer = nn.RMSProp(gan_model.trainable_params(), learning_rate=gan_model_lr)
g_grad_fn = ms.value_and_grad(g_forward_fn, None, g_optimizer.parameters, has_aux=True)
d_optimizer.update_parameters_name('optim_d')
g_optimizer.update_parameters_name('optim_g')

## Visualization

Visualization of training result and prediction result.


In [16]:
y1 = np.linspace(-1, 1, 128)
x1 = np.linspace(-1, 1, 128)
xx, yy = np.meshgrid(x1, y1)  # xy_data.new

def sample_images(epoch):  # plot_n = 5
    idx_train = np.random.randint(0, sample_num_train, plot_n)
    idx_test = np.random.randint(0, sample_num_test, plot_n)
    latent_z_input_train = Tensor(np.random.normal(0, 1, (
        plot_n, latent_z_n_channel, merge_n_imgs, merge_n_imgs)), dtype=ms.float32)
    latent_z_input_test = Tensor(np.random.normal(0, 1, (
        plot_n, latent_z_n_channel, merge_n_imgs, merge_n_imgs)), dtype=ms.float32)
    gen_u_I, gen_u_II, gen_u_III, gen_u_IV, _, _, _, _, _, _ = gan_model([
        Tensor(loader[idx_train][4]),
        Tensor(loader[idx_train][5]),
        Tensor(loader[idx_train][6]),
        latent_z_input_train])
    gen_u_I_, gen_u_II_, gen_u_III_, gen_u_IV_, _, _, _, _, _, _ = gan_model([
        Tensor(cp_fluc_test[idx_test, :, :, :]),
        Tensor(Re_c_test[idx_test, :]),
        Tensor(scaling_input_test[idx_test, :]),
        latent_z_input_test])
    del _
    plot_images(epoch,
                velocity_part=0,
                gen_u_I=gen_u_I,
                gen_u_II=gen_u_II,
                gen_u_III=gen_u_III,
                gen_u_IV=gen_u_IV,
                sel_index=idx_train,
                is_train=True)
    plot_images(epoch,
                velocity_part=1,
                gen_u_I=gen_u_I,
                gen_u_II=gen_u_II,
                gen_u_III=gen_u_III,
                gen_u_IV=gen_u_IV,
                sel_index=idx_train,
                is_train=True)
    plot_images(epoch,
                velocity_part=0,
                gen_u_I=gen_u_I_,
                gen_u_II=gen_u_II_,
                gen_u_III=gen_u_III_,
                gen_u_IV=gen_u_IV_,
                sel_index=idx_test,
                is_test=True)
    plot_images(epoch,
                velocity_part=1,
                gen_u_I=gen_u_I_,
                gen_u_II=gen_u_II_,
                gen_u_III=gen_u_III_,
                gen_u_IV=gen_u_IV_,
                sel_index=idx_test, is_test=True)

def plot_images(epoch, velocity_part, gen_u_I, gen_u_II, gen_u_III, gen_u_IV, sel_index, is_train=False, is_test=False):
    # including ur,vr,gradur, and grad_vr
    # velocity_part = 0 for u and 1 for v
    velocity_name = list(['u', 'v'])
    _, ax = plt.subplots(5, 8, figsize=(20, 16))
    if is_train:
        index = 0
        for i in range(0, 5, 1):
            ax[i, 0].contourf(xx, yy,
                              gen_u_I[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 0].axis('off')
            ax[i, 1].contourf(xx, yy, loader[sel_index[index]][0][velocity_part, :, :], cmap='coolwarm')
            ax[i, 1].axis('off')
            ax[i, 2].contourf(xx, yy,
                              gen_u_II[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 2].axis('off')
            ax[i, 3].contourf(xx, yy, loader[sel_index[index]][1][velocity_part, :, :], cmap='coolwarm')
            ax[i, 3].axis('off')
            ax[i, 4].contourf(xx, yy,
                              gen_u_III[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 4].axis('off')
            ax[i, 5].contourf(xx, yy, loader[sel_index[index]][2][velocity_part, :, :], cmap='coolwarm')
            ax[i, 5].axis('off')
            ax[i, 6].contourf(xx, yy,
                              gen_u_IV[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 6].axis('off')
            ax[i, 7].contourf(xx, yy, loader[sel_index[index]][3][velocity_part, :, :], cmap='coolwarm')
            ax[i, 7].axis('off')

            index = index + 1
        plt.savefig('training_results/images/imgs_train_' + velocity_name[velocity_part] + '%d.png' % (
            epoch))
        plt.close()
    if is_test:
        index = 0
        for i in range(0, 5, 1):
            ax[i, 0].contourf(xx, yy,
                              gen_u_I[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 0].axis('off')
            ax[i, 1].contourf(xx, yy,
                              u_r10_test[sel_index[index], velocity_part, :, :].reshape(n_imgs, n_imgs),
                              cmap='coolwarm')
            ax[i, 1].axis('off')
            ax[i, 2].contourf(xx, yy,
                              gen_u_II[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 2].axis('off')
            ax[i, 3].contourf(xx, yy,
                              u_r5_test[sel_index[index], velocity_part, :, :].reshape(n_imgs, n_imgs),
                              cmap='coolwarm')
            ax[i, 3].axis('off')
            ax[i, 4].contourf(xx, yy,
                              gen_u_III[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 4].axis('off')
            ax[i, 5].contourf(xx, yy,
                              u_r3_test[sel_index[index], velocity_part, :, :].reshape(n_imgs, n_imgs),
                              cmap='coolwarm')
            ax[i, 5].axis('off')
            ax[i, 6].contourf(xx, yy,
                              gen_u_IV[index, velocity_part, :, :].reshape(n_imgs, n_imgs).asnumpy(),
                              cmap='coolwarm')
            ax[i, 6].axis('off')
            ax[i, 7].contourf(xx, yy,
                              u_r1_test[sel_index[index], velocity_part, :, :].reshape(n_imgs, n_imgs),
                              cmap='coolwarm')
            ax[i, 7].axis('off')
            index = index + 1
        plt.savefig('training_results/images/imgs_test_' + velocity_name[velocity_part] + '%d.png' % (
            epoch))
        plt.close()
    plt.cla()
    plt.close('all')

## Model Training

Predict is performed during model training. The accuracy on the training dataset, validation dataset, and test dataset, and the visualization of the predict results are saved every sample_interval epochs. Simultaneously, users could save checkpoint file every sample_interval steps.


In [17]:
critic_model.set_train()
gan_model.set_train()

for epoch in range(epochs):
    start = time.perf_counter()
    ave_d_loss_train = list()
    ave_g_loss_train = list()
    for _, (u_r10_train, u_r5_train, u_r3_train, u_r1_train, cp_fluc_train, Re_c_train, scaling_input_train) in (
            enumerate(tqdm(train_dataset))):
        ave_g_loss_train_ncritic = list()
        latent_z_input_train = ops.normal(
            (batch_size, latent_z_n_channel, merge_n_imgs, merge_n_imgs),
            0, 1, )
        input = [u_r10_train, u_r5_train, u_r3_train, u_r1_train,
                 cp_fluc_train, Re_c_train, scaling_input_train, latent_z_input_train]
        g_true = [u_r10_train, u_r5_train, u_r3_train, u_r1_train]
        train_step(g_true, input)
    ave_d_loss_train = np.mean(ave_d_loss_train, axis=0)
    ave_g_loss_train = np.mean(ave_g_loss_train, axis=0)

    # predict the d_loss and g_loss in validation
    idx_ = np.random.randint(0, sample_num_val, batch_size)
    latent_z_input_validation = Tensor(np.random.normal(0, 1, (
        batch_size, latent_z_n_channel, merge_n_imgs, merge_n_imgs)), dtype=ms.float32)
    batch_d_loss_val, batch_g_loss_val = define_evaluation(
        u_r10_validation, u_r5_validation, u_r3_validation, u_r1_validation,
        cp_fluc_validation, Re_c_validation, scaling_input_validation, latent_z_input_validation, idx_)

    # predict the d_loss and g_loss in testing
    idx__ = np.random.randint(0, sample_num_test, batch_size)
    latent_z_input_test = Tensor(np.random.normal(0, 1, (
        batch_size, latent_z_n_channel, merge_n_imgs, merge_n_imgs)), dtype=ms.float32)
    batch_d_loss_test, batch_g_loss_test = define_evaluation(
        u_r10_test, u_r5_test, u_r3_test, u_r1_test,
        cp_fluc_test, Re_c_test, scaling_input_test, latent_z_input_test, idx__)

    end = time.perf_counter()
    time_consumption = (end - start) // 60  # covert s. to min.
    # Plot the progress and index[0] is the weighted loss
    print(
        "%d:[%d min] [D:%.2f, %.2f, %.2f, %.2f, %.2f] [G:%.2f, %.2f, %.2f, %.2f, %.2f] "
        "[L2_train:%.5f, %.5f, %.5f, %.5f] [L2_val:%.5f, %.5f, %.5f, %.5f] [L2_test:%.5f, %.5f, %.5f, %.5f]" %
        (epoch, time_consumption,
         ave_d_loss_train[0],
         ave_d_loss_train[1] + ave_d_loss_train[5] + ave_d_loss_train[9],
         ave_d_loss_train[2] + ave_d_loss_train[6] + ave_d_loss_train[10],
         ave_d_loss_train[3] + ave_d_loss_train[7] + ave_d_loss_train[11],
         ave_d_loss_train[4] + ave_d_loss_train[8] + ave_d_loss_train[12],
         ave_g_loss_train[0],
         ave_g_loss_train[1], ave_g_loss_train[2], ave_g_loss_train[3], ave_g_loss_train[4],
         ave_g_loss_train[5], ave_g_loss_train[6], ave_g_loss_train[7], ave_g_loss_train[8],
         batch_g_loss_val[5], batch_g_loss_val[6], batch_g_loss_val[7], batch_g_loss_val[8],
         batch_g_loss_test[5], batch_g_loss_test[6], batch_g_loss_test[7], batch_g_loss_test[8]))

    losslog.append(np.concatenate(  # 60
        ([ave_d_loss_train[i] for i in range(13)],  # 13
         [batch_d_loss_val[i] for i in range(13)],  # 13
         [batch_d_loss_test[i] for i in range(13)],  # 13
         [ave_g_loss_train[i] for i in range(13)],  # 13
         [batch_g_loss_val[i] for i in range(13)],  # 13
         [batch_g_loss_test[i] for i in range(13)])))  # 13

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        sample_images(epoch)
        ms.save_checkpoint(gan_model, "training_results/model/gan_%d" % epoch + ".ckpt")
        ms.save_checkpoint(critic_model, "training_results/model/critic_%d" % epoch + ".ckpt")
        np.savetxt('training_results/loss/loss.txt', losslog, fmt='%.4f')
    del ave_d_loss_train, ave_g_loss_train, batch_d_loss_val, batch_g_loss_val, batch_d_loss_test, batch_g_loss_test
    gc.collect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:06<00:00, 33.33s/it]


0:[1 min] [D:38.91, 0.99, 0.96, 0.96, 0.97] [G:17944.48, 0.01, 0.11, 0.00, -0.10] [L2_train:0.46022, 0.48499, 0.47539, 0.43928] [L2_val:0.39236, 0.37077, 0.35429, 0.25296] [L2_test:0.40963, 0.35561, 0.35587, 0.25345]


 50%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                          | 1/2 [00:31<00:31, 31.67s/it]


KeyboardInterrupt: 

## **Result visualization**

The predicted and true values of the velocity fields at various scales in the U and V directions generated by the network in the train dataset under 300 epochs of training are shown in the following figure:

<img src="images/train_u.png" style="zoom:25%" align="center"> <img src="images/train_v.png" style="zoom:25%" align="center">

The predicted and true values of the velocity fields at various scales in the U and V directions generated by the network in the test dataset under 300 epochs of training are shown in the following figure:

<img src="images/test_u.png" style="zoom:25%" align="center"> <img src="images/test_v.png" style="zoom:25%" align="center">

Here, each row give one sample. And one sample give eight figures, each group has two figures including generated flow field at left and the true flow field at right, and the scale decreases from left to right among each group.
